### Count pairs of two consequent words where the first word is "narodnaya" using Apache spark

In [1]:
#! /usr/bin/env python
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
    except ValueError as e:
        return []
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    return words

wiki = sc.textFile(__dir__ + "/articles-part", 16).map(parse_article)

In [9]:
def format_pairs(l):
    li = []
    
    for i in range(len(l)-1):
        if(l[i] == "narodnaya"):
            tp = '{}_{}'.format(l[i], l[i+1])
            li.append((tp, 1))
    
    return li    

In [10]:
#Filter
interm = wiki.map(lambda l: [w.lower() for w in l])
interm = interm.flatMap(lambda l: format_pairs(l))
interm = interm.filter(lambda l: l != [])
interm = interm.reduceByKey(lambda x,y: x + y)
#interm = interm.sortByKey()

In [15]:
result = interm.collect()
for pair in result:
    print "%s\t%d" % (pair[0], pair[1])

narodnaya_gazeta	1
narodnaya_volya	9
